# Aggregating data about platforms from different sources
* 'platforms_hltb.txt' and 'popular_platforms_hltb.txt': listing respectively all popular available platforms from **https://howlongtobeat.com/stats**
* 'platforms_wikipedia.csv': table from **https://en.wikipedia.org/wiki/Home_video_game_console#Released_systems**
* 'console_sales_vgchartz.csv': table from **https://www.vgchartz.com/charts/platform_totals/Hardware.php/**

HowLongToBeat website is the main source of data. We tried to match data from the other sources based on the platform name.


In [256]:
import os
import pandas as pd
from pathlib import Path
from datetime import datetime

In [257]:
#importing all data
absPath = str(Path(os.path.abspath(os.getcwd())).absolute())
datasetsPath = os.path.join(absPath, "raw_datasets\platforms")

popular = pd.read_csv(datasetsPath + '\popular_platforms_hltb.txt',header=None,names=['Platform'])
platforms = pd.read_csv(datasetsPath + '\platforms_hltb.txt',header = None,names=['Platform'])
wikipedia = pd.read_csv(datasetsPath + '\platforms_wikipedia.csv')
sales = pd.read_csv(datasetsPath + '\console_sales_vgchartz.csv')

In [258]:
#adding popularity
column_popular = []
for i in range(len(popular)):
    column_popular.append(True)
popular['Popular']  = column_popular

platforms = platforms.join(popular.set_index("Platform"), on="Platform")


In [259]:
#adding wikipedia data
wikipedia=wikipedia.drop(columns='Units sold')
wikipedia = wikipedia.rename({'Name': 'Platform'}, axis=1)
platforms = pd.merge(platforms, wikipedia,on='Platform',how='left')

In [260]:
#adding sales data
separator = ' ('
acronym = {}
sales['Acronym']=''

#platforms have names like "PlayStation 3 (PS3)". Match on "Playstation 3" to merge and keep "PS3" as acronym.
for i in platforms['Platform']:
    for e in range(len(sales['Platform'])):
        value = sales.loc[e,'Platform']
        if i == value.split(separator)[0]:
            sales.loc[e,'Platform'] = value.split(separator)[0]
            sales.loc[e,'Acronym'] = value.split(separator)[1][:-1]

platforms = pd.merge(platforms, sales, on='Platform', how='left')

In [261]:
#process dates
def parse_date(datestring):
    try:
        return datetime.strptime(datestring, '%B %d, %Y').date()
    except:
        try:
            return datetime.strptime(datestring, '%B %Y').date()
        except:
            try:
                return datetime.strptime(datestring, '%Y').date()
            except:
                return



for i in range(len(platforms)):
    platforms.loc[i,"Release date"] = parse_date(platforms.loc[i,"Release date"])

platforms.to_csv('cleaned_datasets\platforms.csv',index=False)
platforms

,Platform,Popular,Release date,Manufacturer,CPU,"""Bits""",North America,Europe,Japan,Rest of World,Global,Acronym
0,3DO,NaN,1993-10-04,Panasonic/Sanyo (Japan)/GoldStar (South Korea),RISC CPU ARM60 based on ARM architecture @ 12....,32-bit,NaN,NaN,NaN,NaN,NaN,NaN
1,Acorn Archimedes,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amazon Luna,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Amiga,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Amiga CD32,NaN,1993-09-17,Commodore (Canada),Motorola 68EC020@ 14.18 MHz (PAL) 14.32 MHz (N...,32-bit,NaN,0.10,NaN,NaN,0.10,CD32
...,...,...,...,...,...,...,...,...,...,...,...,...
91,Xbox One,True,2013-11-22,Microsoft (U.S.),Custom 1.75 GHz AMD 8-core APU (2 quad-core Ja...,64-bit (CPU),31.61,13.01,0.12,6.54,51.28,XOne
92,Xbox Series X/S,True,2020-11-10,Microsoft (U.S.),"Custom 8-core AMD Zen 2;Series X: 3.8 GHz, 3.6...",64-bit (CPU),8.95,4.76,0.37,3.42,17.51,XS
93,Zeebo,NaN,2009-05-25,Zeebo Inc. (U.S.) / TecToy (Brazil),ARM11 / QDSP-5 in Qualcomm MSM SoC running at ...,32-bit (CPU),NaN,NaN,NaN,NaN,NaN,NaN
94,ZX Spectrum,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
